In [1]:
%pylab inline
import pandas as pd
import numpy as np
import fmt

Populating the interactive namespace from numpy and matplotlib


# Homework Set 7

This homework is to price [synthetic CDO](https://en.wikipedia.org/wiki/Synthetic_CDO) using the one factor Gaussian Copula model. 

A synthetic CDO consists of $n$ CDS, the total loss of the portfolio is defned as:

$$ l(t) = \sum_i^n w_i \tilde {\mathbb{1}}_i(t) (1-r_i(t)) $$

where $w_i$ and $r_i(t)$ are the notional weights and recovery rate of the i-th name in the portfolio. The notional weighs sum up to 1: $\sum_i w_i = 1 $. The $ \tilde {\mathbb{1}}_i(t) $ is the default indicator of the i-th name defaulted before time $t$, the default probability is therefore $p_i(t) = \mathbb E[\tilde {\mathbb{1}}_i(t) ]$

For the purpose of this homework, we consider a simplified synthetic CDO that has no coupon payments, therefore the PV of a \$1 notional synthetic CDO tranche with maturity $t$, attachment $a$ and detachment $d$ is:

$$ v(a, d) = \frac{d(t)}{d-a} \min\left((l(t) - a)^+, d-a\right) $$

where $d(t)$ is the discount factor.

The following are the parameters to the synthetic CDO, and a straight forward Monte Carlo pricer:

In [2]:
n = 125
t = 5.
defProbs = 1 - exp(-(np.random.uniform(size=n)*.03)*t)
recovery = 0.4*np.ones(n)
w = 1./n*np.ones(n)
rho = 0.5
discf = .9
npath = 1000

# a list of attachements and detachements, they pair up by elements
attachements = np.array([0, .03, .07, .1, .15, .3])
detachements = np.array([.03, .07, .1, .15, .3, .6])

#portfolio expected loss
el = np.sum(w*defProbs*(1-recovery))
print("portfolio expected loss is ", el)

portfolio expected loss is  0.046855346002906896


In [3]:
from scipy.stats import norm

class CDO(object) :
    def __init__(self, w, defProbs, recovery, a, d) :
        self.w = w/np.sum(w)
        self.p = defProbs
        self.rec = recovery
        self.rho = rho
        self.a = a
        self.d = d

    def drawDefaultIndicator(self, z, rho) :
        '''return a list of default indicators given common factor z, using one factor Gaussian Copula
        '''
        e = np.random.normal(size=np.shape(self.p))
        x = z*np.sqrt(self.rho) + np.sqrt(1-self.rho)*e
        return np.less(norm.cdf(x), self.p)

    def portfolioLoss(self, defIndicator) :
        '''compute portfolio loss given default indicators'''
        return np.sum(defIndicator*self.w*(1-self.rec))

    def tranchePV(self, portfLoss, discf) :
        '''compute tranche PV from portfolio loss
        Args:
            portfLoss: the total portfolio loss
            discf: discount factor
        Returns:
            tranche PVs'''
        
        sz = self.d - self.a
        return discf/sz*np.minimum(np.maximum(portfLoss - self.a, 0), sz)

    def drawPV(self, z, rho, discf) :
        ''' compute PV and portfolio Loss conditioned on a common factor z'''
        di = self.drawDefaultIndicator(z, rho)
        pfLoss = self.portfolioLoss(di)
        return self.tranchePV(pfLoss, discf), pfLoss
    
    
cdo = CDO(w, defProbs, recovery, attachements, detachements)

In [4]:
## price the tranches using simulation
def simCDO(cdo, rho, disc, paths) :
    zs = np.random.normal(size=[paths])
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs:
        thisPV, _ = cdo.drawPV(z, rho, discf)
        pv += thisPV
        pv2 += thisPV*thisPV
        
    v = pv/paths
    var = pv2/paths - v**2
    return pv/paths, np.sqrt(var/paths)

In [5]:
pv_0, err_0 = simCDO(cdo, rho, discf, npath)
df = pd.DataFrame(np.array([cdo.a, cdo.d, pv_0, err_0]), index=['Attach', 'Detach', 'PV', 'MC err'])

fmt.displayDF(df, fmt='4g')

,0,1,2,3,4,5
Attach,0,0.03,0.07,0.1,0.15,0.3
Detach,0.03,0.07,0.1,0.15,0.3,0.6
PV,0.4753,0.2519,0.1562,0.1024,0.0375,0.003578
MC err,0.01239,0.01192,0.01034,0.008636,0.004882,0.001088


## Problem 1

Modify the simCDO function to implement the following variance reduction techniques, and show whether the technique is effective:

For this homework, we only apply the variance reduction in the common market factor $z$, you should not change the random number $e$ that were drew with in the drawDefaultIndicator function, i.e., only modify the simCDO code, re-use but do not modify the CDO class. Unless explicitly mentioned, keep the simulation path the same as the base case above.

1. anti-thetic variate, reduce the number of paths by half to account for the 2x increase in computation
1. importance sampling, shift $z$ by -1
1. sobol sequence

Compute the **variance** reduction factor for each technique, and comment on the effectiveness of these variance reduction techniques.

### Anti-thetic variate:

In [6]:
# price the simCDO with anti-thetic variate
def simCDO_antiVar(do, rho, disc, paths):
    newPaths = paths // 2
    zs = np.random.normal(size=[newPaths])
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs:
        # anti-thetic variate part
        # firstly price the simCDO with generated random Gaussion
        thisPV1, _ = cdo.drawPV(z, rho, disc)
        # then price the simCDO with the mirror of random Gaussian
        thisPV2, _ = cdo.drawPV(-z, rho, disc)
        # take the average
        thisPV = (thisPV1 + thisPV2) / 2.
        pv += thisPV
        pv2 += thisPV*thisPV
    
    v = pv/newPaths
    var = pv2/newPaths - v**2  
    return pv/newPaths, np.sqrt(var/newPaths)

In [7]:
np.random.seed(11078211)
pv_0_antiVar, err_0_antiVar = simCDO_antiVar(cdo, rho, discf, npath)
# note that anti-thetic variate only has paths half of basic Monte Carlo's paths
df_antiVar = pd.DataFrame(np.array([cdo.a, cdo.d, pv_0_antiVar, err_0_antiVar, (err_0/err_0_antiVar)**2*2]), 
                  index=['Attach', 'Detach', 'PV', 'MC err', 'Variance Reduction factor'])

fmt.displayDFs(df_antiVar, headers = ['Anti-thetic Variate'], fmt='4g')

The result and variance reduction factor of applying Anti-thetic Variate to price synthetic CDO is showed above. <br>

From the result, we may know that in general, Anti-thetic Variate is an effective variance reduction method, since all variance reduction factors are significantly greater than 1.  <br>

Also, the effectiveness of this method decreases as the attachment and detachment value increases. It may be because that lower attachment and detachment value will lead to lower correlation between the result computed by a random gaussion value and its mirror.

### Importance sampling

In [8]:
# price the simCDO with importance sampling
def simCDO_iptSpl(cdo, rho, disc, paths):
    zs = np.random.normal(size=[paths])
    # importance sampling part
    drift = -1
    # price the simCDO with generated random Gaussion with drift
    zs_q = zs + drift
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs_q:
        thisPV, _ = cdo.drawPV(z, rho, disc)
        # multiplying dP/dQ, the coefficient of measure changes
        thisPV = thisPV * np.exp(-drift*z + .5*drift*drift)
        pv += thisPV
        pv2 += thisPV*thisPV
    
    v = pv/paths
    var = pv2/paths - v**2  
    return pv/paths, np.sqrt(var/paths)

In [9]:
np.random.seed(11078211)
pv_0_iptSpl, err_0_iptSpl = simCDO_iptSpl(cdo, rho, discf, npath)
df_impSpl = pd.DataFrame(np.array([cdo.a, cdo.d, pv_0_iptSpl, err_0_iptSpl, (err_0/err_0_iptSpl)**2]), 
                  index=['Attach', 'Detach', 'PV', 'MC err', 'Variance Reduction factor'])

fmt.displayDFs(df_impSpl, headers = ['Importance Sampling'], fmt='4g')

The result and variance reduction factor of applying Importance Sampling to price synthetic CDO is showed above. <br>

From the result, we may know that in general, Importance Sampling is an effective variance reduction method, since all variance reduction factors are significantly greater than 1.  <br>

Also, the effectiveness of this method increases as the attachment and detachment value increases. So it is kind of a complement method of Anti-thetic Variate in this application. <br>

We can apply Anti-thetic Variate for lower attachment and detachment value, and apply Importance Sampling for higher one.

### Sobol sequence

In [10]:
from sobol_seq import i4_sobol_generate as sobol

# price the simCDO with sobol sequence
def simCDO_sobSeq(cdo, rho, disc, paths):
    # sobol sequence part
    # generate the low discrepancy sequence (sobol sequence) of standard random Gaussion
    zs = norm.ppf(np.array(sobol(1, paths, 0)[:, 0]))
    # use this sequence to do the pricing
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs:
        thisPV, _ = cdo.drawPV(z, rho, discf)
        pv += thisPV
        pv2 += thisPV*thisPV
        
    v = pv/paths
    var = pv2/paths - v**2
    return pv/paths, np.sqrt(var/paths)

In [11]:
np.random.seed(11078211)
pv_0_sobSeq, err_0_sobSeq = simCDO_sobSeq(cdo, rho, discf, npath)
df_sobSeq = pd.DataFrame(np.array([cdo.a, cdo.d, pv_0_sobSeq, err_0_sobSeq, (err_0/err_0_sobSeq)**2]), 
                  index=['Attach', 'Detach', 'PV', 'MC err', 'Variance Reduction factor'])

fmt.displayDFs(df_sobSeq, headers = ['Sobol Sequence'], fmt='4g')

The result and variance reduction factor of applying Sobol Sequence to price synthetic CDO is showed above. <br>

From the result, we may know that Sobol Sequence is an ineffective variance reduction method here, since all variance reduction factors are close to 1.  <br>